설명
- 각 코드의 작성자는 마크다운 셀의 가장 끝부분에 (이름)으로 작성함
- 만일 이름이 쓰여 있지 않은 경우, 바로 위와 작성자가 동일하거나, 대주제의 작성자와 동일함을 의미

# 2개 데이터 병합 (김묘경)
- 사용 데이터: ais 신호 정보와 선박 정보
- 공통키: vsl id

In [ ]:
import pandas as pd
import numpy as np
ais_info = pd.read_csv('./ais_for_waiting_area_add_draft.csv')
print(ais_info.shape)

ship_info = pd.read_csv('./VSL_INFO 결측치 정리.csv')
print(ship_info.shape)

from datetime import datetime, timedelta
ais_info['VSL_TIMESTAMP'] = pd.to_datetime(ais_info['VSL_TIMESTAMP'],format='%Y-%m-%d %H:%M:%S')

In [ ]:
# ais + ship
df = pd.merge(ais_info,ship_info)
df['TRIP'] = list(np.full(df.shape[0],-1.0))
df.columns

In [ ]:
df.to_csv('ais_ship_merge.csv', index=False,encoding='utf-8-sig')

# 근접 항구간의 항해 제거 (김묘경) 

## 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
# 항구의 좌표가 담긴 csv
port_df = pd.read_csv('./240603_all_port.csv') # ,sep = '\t'
port_df.shape

In [ ]:
# ais+선박정보 데이터 csv
before_df = pd.read_csv('./ais_ship_merge.csv') 
print(before_df.shape)
before_df['VSL_TIMESTAMP'] = pd.to_datetime(before_df['VSL_TIMESTAMP'],format='%Y-%m-%d %H:%M:%S')
print(before_df.info())

## 근접항 여부 파악 및 제거하여 저장

In [ ]:
def Drop_Near(port_info,merged_df,near):
    temp_list = []
    for i in ['CNSHA', 'SGSIN', 'KRBNP', 'HKHKG', 'KRPUS', 'KRINC']: 
        for j in range(1, port_df.shape[0]-1):
            tt1 = abs(port_info[port_info['port']==i]['lat'].values[0] - port_info.iloc[j]['lat'])
            tt2 = abs(port_info[port_info['port']==i]['lon'].values[0] - port_info.iloc[j]['lon'])
            if tt1==0 and tt2==0: 
                continue
            elif tt1<=near and tt2<=near:
                temp_list.append((port_df.iloc[j]['port'],port_df[port_df['port']==i]['port'].values[0]))
    print('근접항 수:',len(temp_list)) 
    #print(temp_list)

    temp_100 = pd.DataFrame(columns=merged_df.columns)
    for i in temp_list: 
        a = merged_df[merged_df['FROM_PORT_CD']==i[0]]
        a = a[a['TO_PORT_CD']==i[1]]
        temp_100 = pd.concat([temp_100,a])
    new_df = merged_df.drop(index=temp_100.drop_duplicates().index.tolist())
    print('제거 완료 후 남은 데이터:',new_df.shape)
    # 파일로 저장 
    new_df.to_csv(f'./keep_csv/{near}_drop_near_ports.csv', index=False,encoding='utf-8-sig')

In [ ]:
#near_list = [0.4,2,2.4,3]
near_list = [2.2,2.6,2.8,3.2]
for find_near in near_list:
    Drop_Near(port_df,before_df,find_near)

# 항해별로 분리하기 (김묘경)
기준 : ais 신호가 도착항 근처에서 출발항 근처로 이동하게 되는 순간 

## ais 신호 간 좌표 비교 함수

In [ ]:
def Neareast_Check(standard,loc,num=1):
    condition1 = standard-num <= loc
    #print(condition1)
    condition2 = loc <= standard+num
    #print(condition2)
    return condition1 and condition2

## 항해 분리

In [ ]:
def CutTrip(from_port_info,to_port_info,before_cut_df,real_df,trip, num=1):
  for i in range(before_cut_df.shape[0]-1): # 1선박, 1출발항, 1도착항 df에 대해...
    first = before_cut_df.iloc[i]
    first_index = pd.DataFrame(before_cut_df.iloc[i]).T.index[0]
    second = before_cut_df.iloc[i+1]
    second_index = pd.DataFrame(before_cut_df.iloc[i+1]).T.index[0]


    #first_condition1 = to_port_info['lat'].values[0]-1 <= first['LAT'] <= to_port_info['lat'].values[0]+1
    #first_condition2 = to_port_info['lon'].values[0]-1 <= first['LON'] <= to_port_info['lon'].values[0]+1

    #second_condition1 = from_port_info['lat'].values[0]-1 <= second['LAT'] <= from_port_info['lat'].values[0]+1
    #second_condition2 = from_port_info['lon'].values[0]-1 <= second['LON'] <= from_port_info['lon'].values[0]+1

    a = Neareast_Check(to_port_info['lat'].values[0],first['LAT'],num) and Neareast_Check(to_port_info['lon'].values[0],first['LON'],num)
    b = Neareast_Check(from_port_info['lat'].values[0],second['LAT'],num) and Neareast_Check(from_port_info['lon'].values[0],second['LON'],num)
    last_condition = a and b

    #if first_condition1 and first_condition2 and second_condition1 and second_condition2 :
    if last_condition:
      # print(to_port['lat'].values[0],first['LAT'],'/',to_port['lon'].values[0],first['LON'])

      real_df.at[first_index,'TRIP'] = trip
      trip+=1
      #print(i,'.',first_index)
      if i >= before_cut_df.shape[0]-2:
        real_df.at[second_index,'TRIP'] = trip
    else:
      real_df.at[first_index,'TRIP'] = trip
      if i >= before_cut_df.shape[0]-2:
        real_df.at[second_index,'TRIP'] = trip

  return real_df,trip


  # real_df['TRIP'].value_counts()

## 그룹핑된 선박 정보를 받아, 추가 그룹핑 및 위의 함수들을 진행 
<전체 방식> 
1. 위에서 근접 항해를 제거한 데이터 사용 
2. 항구좌표 데이터, 선박별로 그룹핑된 데이터, 원본 데이터, 가까움의 기준 거리 사용
3. 선박별로 그룹핑 된 데이터에 대해 출발항 별로 그룹핑하고, 도착항 별로 그룹핑 진행
4. 그룹핑 전의 데이터 프레임에 TRIP 컬럼 추가
5. 항해 번호는 하나의 선박 당 1부터 시작하도록 함 

In [ ]:
def Grouped_to_FROM(port_info,grouped_df,before_add_trip_df,num=1):
  trip_count = 1 # 각 선박의 trip 수

  # 출발항별로 그룹핑
  one_ship_from_ports = grouped_df.groupby('FROM_PORT_CD')
  for from_port, one_from_df in one_ship_from_ports:
    from_port_info = port_info[port_info['port']==from_port] # 출발항의 정보

    # 도착항별로 그룹핑
    one_ship_to_ports = one_from_df.groupby('TO_PORT_CD')
    for to_port, one_to_df in one_ship_to_ports:
      to_port_info = port_info[port_info['port']==to_port] # 도착항의 정보
      one_to_df.sort_values('VSL_TIMESTAMP',inplace=True)
      before_add_trip_df,trip_count = CutTrip(from_port_info,to_port_info,one_to_df,before_add_trip_df,trip_count, num)
      trip_count+=1
  return before_add_trip_df

# 선박 이름별로 그룹핑된 선박

# 그룹핑 전의 데이터 프레임 필요함 , 해당 데이터 프레임에 TRIP 컬럼 추가 ()
# df =
# df['TRIP'] = list(np.full(df.shape[0],-1.0))

# Grouped_to_FROM(grouped_df,before_grouped,before_add_trip_df)

## 실제 분리 실행 

### 가까움의 기준을 바꿔가며 분리 실행 및 저장 

#### 항해 분리 및 저장
- 근접항 제거에서 저장했던 데이터를 불러와 사용

In [ ]:
# near_list = [0.4,2,2.4,3]
near_list = [2.2,2.6,2.8,3.2]
for find_near in near_list:
    df = pd.read_csv(f'./keep_csv/{find_near}_drop_near_ports.csv')
    grouped_ships = df.groupby('VSL_ID')
    save_df = pd.DataFrame(columns=df.columns)
    count = 1
    for name, one_df in grouped_ships:
      one_df.sort_values('VSL_TIMESTAMP',inplace=True)
      save_df = Grouped_to_FROM(port_df,one_df,df,find_near/2)
      count+=1
      if count%100 == 0:
        save_df.to_csv(f'./last_csv/{find_near}_{find_near/2}_add_trip.csv', index=False,encoding='utf-8-sig')
        print(count)
      #cut_trip_df = Grouped_to_FROM(port_df,one_df,df)
      save_df = pd.concat([save_df,cut_trip_df])
   save_df.to_csv(f'./last_csv/{find_near}_{find_near/2}_add_trip.csv', index=False,encoding='utf-8-sig')

#### 각 분리 기준에 따른 항해 수 비교 

In [ ]:
keep_recode = []
for near in near_list: 
    last_recode_df = pd.read_csv(f'./last_csv/{near}_{near/2}_last_recode.csv')
    print(near,'전체 마지막 레코드 수(항해 수):',last_recode_df.shape)
    keep_recode.append(last_recode_df.shape[0])
    #all_loc = [(one_port_df.iloc[i]['LAT'],one_port_df.iloc[i]['LON']) for i in range(one_port_df.shape[0])]
    #all_lat = [one_port_df.iloc[i]['LAT'] for i in range(one_port_df.shape[0])]
    #all_lon = [one_port_df.iloc[i]['LON'] for i in range(one_port_df.shape[0])]
    #break

In [ ]:
#x = np.arange(0,20)
#y = x*2  

x_ticks = near_list #x축의 눈금을 표시할 index값
plt.bar(np.arange(0,14),keep_recode,width=0.5) #label: 막대의 이름
plt.xlabel('drop_near_size')
plt.ylabel('trips')
#plt.title('Bar Chart Example')
plt.xticks(np.arange(0,14),x_ticks)
#plt.yticks(np.arange(0,2700,100))
plt.ylim(2000,2600)
plt.grid(True,axis='y')
#plt.legend() #막대의 이름을 적은 범례를 출력하게 하기 위함
plt.show()

# 신호가 없는 기간이 길면 삭제 (김묘경)
조건 : 도착항의 평균 대기 시간 이상 길면 제거 

## 데이터 파악 

In [ ]:
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder

import numpy as np 
import pandas as pd 

# ['CNSHA', 'SGSIN', 'KRBNP', 'HKHKG', 'KRPUS', 'KRINC']
df = pd.read_csv('2.2_1.1_add_trip.csv') 
print(df.shape) 
df.dropna(inplace=True)
print(df.shape)

## 항해 저장 

In [ ]:
# 항구별 평균 대기 시간 
waithing = {'CNSHA':28, 
 'SGSIN':9.73, 
 'KRBNP':11.07, 
 'HKHKG':5.4, 
 'KRPUS':31, 
 'KRINC':24}
to_ports = ['CNSHA', 'SGSIN', 'KRBNP', 'HKHKG', 'KRPUS', 'KRINC']

In [ ]:
# 전체항이 저장됨 
save_df = pd.DataFrame(columns = df.columns) 
# 도착항별 그룹핑
for to_port, port_df in df.groupby('TO_PORT_CD'):
    # 각 항구별 저장 (문서 저장을 위함)
    one_port_df = pd.DataFrame(columns = port_df.columns)
    # 선박별 그룹핑
    for vsl_id, one_ship in port_df.groupby('VSL_ID'):
        # 각 선박별 저장 (trip값 변경을 위함)
        one_ship_df = pd.DataFrame(columns = one_ship.columns)
        # 항해별 그룹핑
        for name, trip in one_ship.groupby('TRIP'): 
            # 시계열로 변경
            trip['VSL_TIMESTAMP'] = pd.to_datetime(trip['VSL_TIMESTAMP'],format='%Y-%m-%d %H:%M:%S')
            # 시계열 순서대로 정렬
            trip = trip.sort_values('VSL_TIMESTAMP')
            # 1개 항해에 대해 반복
            for i in range(trip.shape[0]-1): 
                # i+1번째 시간 - i번째 시간 
                sub = trip.iloc[i+1]['VSL_TIMESTAMP'] - trip.iloc[i]['VSL_TIMESTAMP']
                # 시간 텀이 sub 이상 존재 => 저장 안함 
                if sub > timedelta(days=waithing[to_port]): 
                    break
                # 마지막 항해라면 
                if i == trip.shape[0]-2:
                    # 정상적 항해 저장
                    one_ship_df = pd.concat([one_ship_df,trip])
        encoded = LabelEncoder().fit_transform(one_ship_df['TRIP'])
        one_ship_df['TRIP'] = encoded
        # 선박별 저장용 df에 추가
        one_port_df = pd.concat([one_port_df,one_ship_df])
    print(f'{to_port}항 완료')
    # 항해 순서값 정렬
    one_port_df.to_csv(f'2.2_{to_port}_drop_outlier.csv', index=False,encoding='utf-8-sig')
    # 항구별 저장용 df에 추가
    save_df = pd.concat([save_df,one_port_df])
save_df.to_csv(f'2.2_all_drop_outlier.csv', index=False,encoding='utf-8-sig')
print('end')

# 기상 데이터 수집
1. 스크래핑으로 수집 
    - 수집 예정 데이터 : 풍속, 해류, 파고
    - 스크래핑 진행 사이트 : https://earth.nullschool.net/
    - 사용한 파일 : ALL_COORDS_in_actual_waiting_df.csv
      - 김정수님이 만들어주신 최초 시점+마지막 시점 병합 데이터 
    
    - 스크래핑 코드 작성자 
      - 풍속 : 김묘경 작성 
      - 해류 : 유지민 작성 
      - 파고 : 곽현수 작성
2. GAPCO 데이터 파일로 수집 (유지민)
   - 수집 데이터 : 수심 

- **주의사항**
- 데이터를 분리하여 3여개 파일로 생성, 기상 수집을 동시 수행 하였기에 불러온 파일 오류 발생 가능성 있음
  - ALL_COORDS_in_actual_waiting_df.csv.csv 파일을 5개로 분리하여 각각 ['first', 'second', 'third', 'fourth', 'fifth'].csv로 저장한 이후 실행하였음 
- 스크래핑 후에도 존재하는 결측치는 분담하여 일일히 채웠음 

## 스크래핑 함수  (곽현수, 김묘경, 유지민) 

### 데이터 호출 

In [ ]:
import pandas as pd 
import numpy as np 

import requests
from selenium.webdriver import Chrome, ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

import time

#페이지가 로드될 때까지 기다리는 시간 (하이퍼 파라미터 사용)
tum = 1.5 #sleep 시간을 일괄적으로 관리하기 위함

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

import warnings
warnings.filterwarnings('ignore')
import re

In [ ]:
df = df.read_csv('ALL_COORDS_in_actual_waiting_df.csv') 
df.shape 

### 단위 변환 함수 (곽현수, 김묘경) 

### 풍속 스크래핑  (김묘경) 

In [ ]:
def GET_Wind_SPEED(driver,year,month,day,hour,minute,lat,lon,PAUSE_TIME=1): 
    try:
        URL = f'https://earth.nullschool.net/#{year}/{month}/{day}/'
        set_time = hour 
        if minute>30: 
            set_time+=1 
        URL = URL+f'{set_time}00Z/wind/surface/level/loc={lon:.3f},{lat:.3f}'
        driver.get(URL)
    
        time.sleep(PAUSE_TIME) #동적으로 생성되는 페이지의 내용이 완성될 때 까지 대기
        get_unit = driver.find_element(By.XPATH,'//*[@id="spotlight-panel"]/div[2]/button/span[1]').text.strip() # 속도 방식 kn, km/h 등 
        last = driver.find_element(By.XPATH,'//*[@id="spotlight-panel"]/div[2]/div').text.split('@')[1].strip()
        wind_speed = Change_Unit_to_kn(last,get_unit)
        #soup = BeautifulSoup(driver.page_source,'lxml')
        #soup.select_one('#spotlight-panel > div:nth-child(2) > div').text.split('@')[1].strip()
    
        # driver.close() #실행되었던 웹드라이버 페이지 종료
        return wind_speed
    except Exception as e:
        print(e,':',URL)
        return -1

### 파고 스크래핑 (곽현수)

In [ ]:
def GET_Wave_Height(driver,year,month,day,hour,minute,lat,lon,PAUSE_TIME=1): 
    try:
        URL = f'https://earth.nullschool.net/#{year}/{month}/{day}/'
        set_time = hour
        if hour < 0 or hour >= 24:
            #print('Wrong hour data')
            pass
        else:
            hour = (hour // 3) * 3

        URL = URL+f'{set_time}00Z/ocean/primary/waves/overlay=significant_wave_height/loc={lon:.3f},{lat:.3f}'
        driver.get(URL)
    
        time.sleep(PAUSE_TIME) #동적으로 생성되는 페이지의 내용이 완성될 때 까지 대기
        get_unit = driver.find_element(By.XPATH,'//*[@id="spotlight-panel"]/div[3]/button/span[1]').text.strip() # 속도 방식 kn, km/h 등 
        last = driver.find_element(By.XPATH,'//*[@id="spotlight-panel"]/div[3]/div').text.strip()
        wave_height = Change_Unit_to_m(last,get_unit)
        if wave_height > 2: 
            print('파고 체크',wave_height,'m :',URL)
        return wave_height
        
    except Exception as e:
        print('추출 불가 :',URL)
        return -1

### 해류 스크래핑 (유지민) 

In [ ]:
def GET_OCEAN_CURRENT(driver,year,month,day,lat,lon,PAUSE_TIME=1): 
    try:
        URL = f'https://earth.nullschool.net/#{year}/{month}/{day}/0000Z/ocean/surface/currents/loc={lon:.3f},{lat:.3f}'
        driver.get(URL)
    
        time.sleep(PAUSE_TIME) #동적으로 생성되는 페이지의 내용이 완성될 때 까지 대기
        get_unit = driver.find_element(By.XPATH,'//*[@id="spotlight-panel"]/div[2]/button/span[1]').text.strip() # 속도 방식 kn, km/h 등
        last = driver.find_element(By.XPATH,'//*[@id="spotlight-panel"]/div[2]/div').text.split('@')[1].strip()
        ocean_current = Change_Unit_to_kn(last,get_unit)
        if ocean_current > 2.5: 
            print('해류 체크',ocean_current,'kn :',URL)
        return ocean_current
    except Exception as e:
        print('추출 불가 :',URL)
        return -1

### 실제 스크래핑 진행 함수  (김묘경)

In [ ]:
def Scrapping(key,data_frame,P_T=1):
    options = Options()
    #options.add_argument('--headless')  # 화면 생성이 일어나지 않음 
    #options.add_argument('--disable-gpu') #GPU 하드웨어 가속을 비활성화. 헤드리스 모드에서 발생할 수 있는 GPU 관련 문제를 피하기 위해 사용
    dd = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    #['VSL_ID', 'TRIP', 'WAITING_ORDER', 'LAT', 'LON', 'WAITING_START','WAITING_END', 'WAITING_TIME(H:M:S)', 'START_LAT', 'START_LON','END_LAT', 'END_LON', 'TO_PORT_CD']
    for i in range(data_frame.shape[0]):
        for j in ['START','END']:
            # try:
            #     if data_frame.iloc[i][j+'_WIND(kn)']==-1 or data_frame.iloc[i][j+'_WIND(kn)']==0:
            #         data_frame.at[i,j+'_WIND(kn)'] = GET_Wind_SPEED(dd,
            #                                                         data_frame.iloc[i]['WAITING_'+j].year,data_frame.iloc[i]['WAITING_'+j].month,data_frame.iloc[i]['WAITING_'+j].day,
            #                                                         data_frame.iloc[i]['WAITING_'+j].hour,data_frame.iloc[i]['WAITING_'+j].minute,
            #                                                         data_frame.iloc[i][j+'_LAT'],data_frame.iloc[i][j+'_LON'],
            #                                                         PAUSE_TIME=P_T)
            # except Exception as e:
            #     #print('기타 에러:',e)
            #     pass
            
            #try:
            if data_frame.iloc[i][j+'_CURRENT(kn)']==-1 or data_frame.iloc[i][j+'_CURRENT(kn)']==0 or data_frame.iloc[i][j+'_CURRENT(kn)']>=2:
                current = GET_OCEAN_CURRENT(dd,
                                            data_frame.iloc[i]['WAITING_'+j].year,data_frame.iloc[i]['WAITING_'+j].month,data_frame.iloc[i]['WAITING_'+j].day,
                                            data_frame.iloc[i][j+'_LAT'],data_frame.iloc[i][j+'_LON'],
                                            PAUSE_TIME=P_T)
                data_frame.at[i,j+'_CURRENT(kn)'] = current 
            #print('해류:',current,'kn')
            # except Exception as e:
            #     #print('기타 에러:',e)
            #     pass
        
            #try:
            if data_frame.iloc[i][j+'_WAVE(m)']==-1 or data_frame.iloc[i][j+'_WAVE(m)']==0 or data_frame.iloc[i][j+'_WAVE(m)']>=1.5:
                wave = GET_Wave_Height(dd,
                                       data_frame.iloc[i]['WAITING_'+j].year,data_frame.iloc[i]['WAITING_'+j].month,data_frame.iloc[i]['WAITING_'+j].day,
                                       data_frame.iloc[i]['WAITING_'+j].hour,data_frame.iloc[i]['WAITING_'+j].minute,
                                       data_frame.iloc[i][j+'_LAT'],data_frame.iloc[i][j+'_LON'],
                                       PAUSE_TIME=P_T)
                data_frame.at[i,j+'_WAVE(m)'] = wave
                #print('파고:',wave,'m')
            # except Exception as e:
            #     #print('기타 에러:',e)
            #     pass

        if i % 30 == 0: 
            print('over',i)
            #print(data_frame.shape)
            data_frame.to_csv(f'cuted_{key}.csv',index=False,encoding='utf-8-sig')
    data_frame.to_csv(f'cuted_{key}.csv',index=False,encoding='utf-8-sig')
    dd.close()
    print(f'{key} 종료')
    print('풍속 결측치:',data_frame[data_frame['START_WIND(kn)']==-1].shape[0]+data_frame[data_frame['END_WIND(kn)']==-1].shape[0],'개')
    print('해류 결측치:',data_frame[data_frame['START_CURRENT(kn)']==-1].shape[0]+data_frame[data_frame['END_CURRENT(kn)']==-1].shape[0],'개')
    print('파고 결측치:',data_frame[data_frame['START_WAVE(m)']==-1].shape[0]+data_frame[data_frame['END_WAVE(m)']==-1].shape[0],'개')
    print('='*50)

### 스크래핑 실행

In [ ]:
for i in ['first', 'second', 'third', 'fourth', 'fifth']: 
    new = pd.read_csv(f'{i}.csv')
    new['WAITING_START'] = pd.to_datetime(new['WAITING_START'],format='%Y-%m-%d %H:%M:%S')
    new['WAITING_END'] = pd.to_datetime(new['WAITING_END'],format='%Y-%m-%d %H:%M:%S')
    print(i,':',new.shape)
    print(new.columns)
    Scrapping(i,new,tum)
    print('='*100)
    #break

## 수심 추출 (유지민) 

In [ ]:
pip install cartopy

In [ ]:
pip install netCDF4

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd 

In [ ]:
import netCDF4 as nc
file = r"C:\Users\0\Desktop\GEBCO_2023\GEBCO_2023.nc"
fdata = nc.Dataset(file)
print(fdata)

In [ ]:
file_path = r"C:\Users\0\Desktop\GEBCO_2023\GEBCO_2023.nc"
ds = xr.open_dataset(file_path)

print(ds)

## 특정 좌표의 수심 구하기

In [ ]:
df1 = pd.read_csv('df_add_trip_waiting_before_arrive.csv')
df1.shape

In [ ]:
def ADD_OCEAN_HEIGHT(latitude,longtitude): 
    # NetCDF 파일 열기 (예제 파일 경로를 사용하세요)
    nc_file = r"C:/Users/user/Desktop/GEBCO_2023/GEBCO_2023.nc"
    dataset = xr.open_dataset(nc_file)
    
    # 고도 데이터 변수 선택 (예: 'elevation' 변수)
    elevation = dataset['elevation']
    
    # 관심 영역의 부분집합 선택
    subset = elevation.sel(lat=slice(20, 50), lon=slice(100, 150))
    
    # 데이터 나누기
    chunks = {'lat': 1000, 'lon': 1000}
    subset = subset.chunk(chunks)
    
    # 데이터 축소 (여기서는 20x 축소)
    subset_reduced = subset.coarsen(lat=20, lon=20, boundary='trim').mean()
    
    # 좌표에서 가장 가까운 셀의 고도 값을 선택
    elevation_value = subset_reduced.sel(lat=latitude, lon=longtitude, method='nearest').compute()

    return elevation_value.values

In [ ]:
import pandas as pd 
import numpy as np
df = pd.read_csv('please_fill_nan4.csv') 
df['START_HEIGHT'] = list(np.full((df.shape[0]),None))
df['END_HEIGHT'] = list(np.full((df.shape[0]), None))
for i in range(df.shape[0]): 
    df.at[i,'START_HEIGHT'] = ADD_OCEAN_HEIGHT(df.at[i,'START_LAT'],df.at[i,'START_LON'])
    df.at[i,'END_HEIGHT'] = ADD_OCEAN_HEIGHT(df.at[i,'END_LAT'],df.at[i,'END_LON'])
print('END')

In [ ]:
display(df.head()) 
print(df.isna().sum()) 

In [ ]:
print(df['HEIGHT'].isna().sum())

In [ ]:
df.to_csv('plz_fill_nan4.csv', index=False, encoding='utf-8-sig')

# 파일에 존재하던 오류 해결 (김묘경)

## 1. 시간의 초가 삭제된 문제 해결 
- 아래 데이터 들은 기상 정보를 추가하여 생성한 데이터를 사용 

In [ ]:
import pandas as pd 
import numpy as np

a = pd.read_csv('aaa(수정).csv') 
print(a.shape)
b = pd.read_csv('bbb(수정).csv')
print(b.shape)
c = pd.read_csv('ccc.csv')  
print(c.shape)
d = pd.read_csv('ddd.csv')
print(d.shape)

temp1 = pd.read_csv('KRINC_ALL_DATA.csv')
print(temp1.shape)
temp2 = pd.read_csv('CNSHA_ALL_DATA.csv')
print(temp2.shape)

In [ ]:
df = pd.concat([a,b,c,d,temp1,temp2])
df.dropna(inplace=True)

df['START_LAT'] = df['START_LAT'].apply(lambda x:np.round(x,6))
df['START_LON'] = df['START_LON'].apply(lambda x:np.round(x,6))
df['END_LAT'] = df['END_LAT'].apply(lambda x:np.round(x,6))
df['END_LON'] = df['END_LON'].apply(lambda x:np.round(x,6))
df['WAIT_LAT'] = df['WAIT_LAT'].apply(lambda x:np.round(x,6))
df['WAIT_LON'] = df['WAIT_LON'].apply(lambda x:np.round(x,6))

df = df.rename(columns={'WAITING_START': 'ws','WAITING_END':'we'})

df.shape

In [ ]:
# 동일한 선박명과 trip값을 가진 경우가 존재하는지 파악 함 
for ship, ship_df in df.groupby('VSL_ID'): 
    for trip, trip_df in ship_df.groupby('TRIP'): 
        if trip_df.shape[0] >=2: 
            print(trip)
            display(trip_df)
print('end')

In [ ]:
former = pd.read_csv('please_fill_nan.csv')
former['START_LAT'] = former['START_LAT'].apply(lambda x:np.round(x,6))
former['START_LON'] = former['START_LON'].apply(lambda x:np.round(x,6))
former['END_LAT'] = former['END_LAT'].apply(lambda x:np.round(x,6))
former['END_LON'] = former['END_LON'].apply(lambda x:np.round(x,6))
former.shape

In [ ]:
re = former[['VSL_ID', 'TRIP','WAITING_TIME(H:M:S)','START_LAT','START_LON','END_LAT','END_LON','WAITING_START','WAITING_END']]
#re.loc[:, 'TRIP'] = re['TRIP'].astype(float)
re['TRIP'] = re['TRIP'].apply(lambda x:float(x)) #np.round(x,1)
re.shape

In [ ]:
df['combined'] = df['VSL_ID'].astype(str) + '_' + df['TRIP'].astype(str) + '_' + df['WAITING_TIME(H:M:S)'].astype(str)
re['combined'] = re['VSL_ID'].astype(str) + '_' + re['TRIP'].astype(str) + '_' + re['WAITING_TIME(H:M:S)'].astype(str)
re = re[['WAITING_START','WAITING_END','combined']]
test = pd.merge(df,re,on='combined',how='left')
print(test.shape)
print(test.isna().sum())

In [ ]:
test.drop('ws',axis=1,inplace=True)
test.drop('we',axis=1,inplace=True)

In [ ]:
df = test[['TO_PORT_CD','VSL_ID', 'TRIP', 'WAITING_TIME(H:M:S)','WAIT_LAT', 'WAIT_LON',
          'WAITING_START','START_LAT','START_LON', 'START_WAVE(m)', 'START_WIND(kn)', 'START_CURRENT(kn)','START_HEIGHT',
          'WAITING_END','END_LAT', 'END_LON', 'END_WAVE(m)', 'END_WIND(kn)', 'END_CURRENT(kn)','END_HEIGHT']]

In [ ]:
df.to_csv('0612_1297with_fix_timestamp.csv')

## 2. 드래프트 추가 
- 동일한 타임스탬프를 가진 때를 찾아 드래프트를 추가하도록 함 

In [ ]:
origin = pd.read_csv('ais_for_waiting_area.csv')
origin.shape

In [ ]:
rr = origin[['FROM_PORT_CD','VSL_ID','LAT','LON','VSL_DRAFT','VSL_SPEED','VSL_TIMESTAMP']]
rr.loc[:,'LAT'] = rr['LAT'].apply(lambda x:np.round(x,6))
rr.loc[:,'LON'] = rr['LON'].apply(lambda x:np.round(x,6))
rr.shape

In [ ]:
ddf = df.copy()
ddf['combined'] = df['VSL_ID'].astype(str) + '_' + df['WAITING_START'].astype(str)
oorigin = rr.copy()
oorigin['combined'] = oorigin['VSL_ID'].astype(str) + '_' + oorigin['VSL_TIMESTAMP'].astype(str)
oorigin = oorigin[['combined','VSL_DRAFT','VSL_SPEED','FROM_PORT_CD']]
oorigin = oorigin.rename(columns={'VSL_DRAFT': 'START_DRAFT','VSL_SPEED':'START_SPEED'})

In [ ]:
just_start = pd.merge(ddf,oorigin,on='combined',how='left')
just_start.shape

In [ ]:
# 동일한 선박명과 trip값을 가진 경우가 존재하는지 파악 함 
for ship, ship_df in just_start.groupby('VSL_ID'): 
    for trip, trip_df in ship_df.groupby('TRIP'): 
        if trip_df.shape[0] >=2: 
            print(trip)
            display(trip_df)
print('end')

In [ ]:
ddf = df.copy()
ddf.loc[:,'combined'] = df['VSL_ID'].astype(str) + '_' + df['WAITING_START'].astype(str)
oorigin2 = rr.copy()
oorigin2.loc[:,'combined'] = oorigin2['VSL_ID'].astype(str) + '_' + oorigin2['VSL_TIMESTAMP'].astype(str)
oorigin2 = oorigin2[['combined','VSL_DRAFT','VSL_SPEED']]
oorigin2 = oorigin2.rename(columns={'VSL_DRAFT': 'END_DRAFT','VSL_SPEED':'END_SPEED'})

In [ ]:
last = pd.merge(just_start,oorigin2,on='combined',how='left')
last.shape

In [ ]:
# 동일한 선박명과 trip값을 가진 경우가 존재하는지 파악 함 
for ship, ship_df in last.groupby('VSL_ID'): 
    for trip, trip_df in ship_df.groupby('TRIP'): 
        if trip_df.shape[0] >=2: 
            print(trip)
            display(trip_df)
print('end')

In [ ]:
saved = last[['FROM_PORT_CD', 'TO_PORT_CD', 'VSL_ID', 'TRIP', 'WAITING_TIME(H:M:S)', 'WAIT_LAT','WAIT_LON',
              'WAITING_START', 'START_LAT', 'START_LON', 'START_WAVE(m)','START_WIND(kn)', 'START_CURRENT(kn)',
              'START_HEIGHT', 'START_DRAFT', 'START_SPEED',
              'WAITING_END', 'END_LAT', 'END_LON', 'END_WAVE(m)', 'END_WIND(kn)', 'END_CURRENT(kn)',
              'END_HEIGHT', 'END_DRAFT', 'END_SPEED']]

In [ ]:
saved.to_csv('0612_data_add_before_add_ship_info.csv',index=False)

## 최종 파일 정보 파악 및 정리 (김묘경) 

### 드래프트 결측치  제거

In [ ]:
import numpy as np 
import pandas as pd 
df = pd.read_csv('0612_data_add_before_add_ship_info.csv')
df.shape

In [ ]:
drop_d_na = df[df['START_DRAFT']!=999]
drop_d_na = drop_d_na[drop_d_na['END_DRAFT']!=999]
drop_d_na.shape

In [ ]:
df['TO_PORT_CD'].value_counts()

In [ ]:
drop_d_na.to_csv('0612_1059_drop_draft_nan.csv',index=False)

### 해류 결측치 파악 

In [ ]:
need_to_add_weather1 = drop_d_na[drop_d_na['START_CURRENT(kn)']==-1]
need_to_add_weather2 = drop_d_na[drop_d_na['END_CURRENT(kn)']==-1]
need_to_add_weather = pd.concat([need_to_add_weather1,need_to_add_weather2],ignore_index=True) 
need_to_add_weather.drop_duplicates(inplace=True) 
need_to_add_weather.shape

### 드래프트 결측치를 제거한 파일에 선박 정보 결합 

In [ ]:
import numpy as np 
import pandas as pd 
ais = pd.read_csv('0612_1059_drop_draft_nan.csv')
print(ais.shape)
ship = pd.read_csv('ais_for_waiting_area_vsl_info.csv')
print(ship.shape)

In [ ]:
ship_n = ship[['VSL_ID','SHIP_TYPE','VSL_LENGTH','VSL_WIDTH','GROSS_TONNAGE']]

In [ ]:
temp = pd.merge(ais,ship_n,how='left') 
temp.shape

In [ ]:
temp.to_csv('0612_before_add_-1.csv')

## 분담하여 결측치를 채운 정보를 결합, 해류 결측 제거 및 저장 

In [ ]:
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)

data_df = pd.read_csv('0612_before_add_-1.csv') 
print(data_df.shape)

a1 = pd.read_excel('0612_1(수정).xlsx')
print(a1.shape)
b2 = pd.read_excel('0612_2(수정).xlsx')
print(b2.shape)
c3 = pd.read_excel('0612_3(수정).xlsx')
print(c3.shape)
d4 = pd.read_excel('0612_4(수정).xlsx')
print(d4.shape)

In [ ]:
temp = data_df[data_df['START_CURRENT(kn)']!=-1]
temp = temp[temp['END_CURRENT(kn)']!=-1]
temp.shape

In [ ]:
df.to_excel('0613_all_data_last.xlsx',index=False)
df.to_csv('0613_all_data_last.csv',index=False)

In [ ]:
df[df['TO_PORT_CD']=='KRPUS'].to_csv('0613_all_data_KRPUS.csv')
df[df['TO_PORT_CD']=='SGSIN'].to_csv('0613_all_data_SGSIN.csv')
df[df['TO_PORT_CD']=='KRBNP'].to_csv('0613_all_data_KRBNP.csv')
df[df['TO_PORT_CD']=='HKHKG'].to_csv('0613_all_data_HKHKG.csv')
df[df['TO_PORT_CD']=='CNSHA'].to_csv('0613_all_data_CNSHA.csv')
df[df['TO_PORT_CD']=='KRINC'].to_csv('0613_all_data_KRINC.csv')

# 실질적 대기 지역 추출 (김정수) 
- 대기 지역 기준: 도착항 위·경도 기준 반경 (± 1.1)도 이내의 범위에서 속도가 3노트 이하로 가장 긴 시간 동안 유지된 구간을 실질적인 대기 구간으로 간주한다.
- 분석 절차: 한 번의 항해에 대해 도착항 근처에서 발생한 대기 기록 중 최장 시간 대기한 경우를 찾아 좌표를 추출한다.
- 최종 목표: 대기지역 좌표기반 군집화 및 시각화

## 필수 실행

In [ ]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')
from datetime import datetime, timedelta

In [ ]:
# 도착항 위·경도 좌표
port_coordinates = {
    'SGSIN': (1.2392, 103.8325),
    'CNSHA': (31.4001, 121.4971),
    'KRPUS': (35.1060, 129.0840),
    'KRBNP': (35.0778, 128.7911),
    'HKHKG': (22.3360, 114.1240),
    'KRINC': (37.4460, 126.6460)
}

# 위·경도 (± 1.1)도 이내 데이터 추출 함수
def vsl_in_radius(df, lat, lon, radius=1.1):
    filtered_df=df[(df['LAT']>=lat-radius)&(df['LAT']<=lat+radius)&
                    (df['LON']>=lon-radius)&(df['LON']<=lon+radius)]
    return filtered_df.reset_index(drop=True)

In [ ]:
# 항구별 데이터 로드
files = {
    'SGSIN': '2.2_SGSIN_drop_outlier.csv',
    'CNSHA': '2.2_CNSHA_drop_outlier.csv',
    'KRPUS': '2.2_KRPUS_drop_outlier.csv',
    'KRBNP': '2.2_KRBNP_drop_outlier.csv',
    'HKHKG': '2.2_HKHKG_drop_outlier.csv',
    'KRINC': '2.2_KRINC_drop_outlier.csv'
}

df_by_port_list = {}
for port_code, file in files.items():
        df_by_port_list[port_code] = pd.read_csv(f'{file}')

# VSL_TIMESTAMP 데이터 타입 변경 및 위도 경도 필터링
for port_code, df_by_port in df_by_port_list.items():
    df_by_port['VSL_TIMESTAMP'] = pd.to_datetime(df_by_port['VSL_TIMESTAMP'])
    lat, lon = port_coordinates[port_code]
    df_by_port = vsl_in_radius(df_by_port, lat, lon)
    df_by_port_list[port_code] = df_by_port

In [ ]:
# 항구별로 반복
all_waiting_data = []

for port_code, df in df_by_port_list.items():
    # 선박별로 그룹화
    grouped_by_vsl = df.groupby('VSL_ID')
    
    # 항해(TRIP)별로 다시 그룹화해서 df_port에 담기
    df_list = []
    for vsl_id, vsl_group in grouped_by_vsl:
        grouped_by_trip = vsl_group.groupby('TRIP')
        for trip_id, trip_group in grouped_by_trip:
            df_list.append(trip_group)
    df_port = pd.concat(df_list)
    # TIMESTAMP의 역순으로 데이터프레임을 정렬
    df_port['VSL_TIMESTAMP'] = pd.to_datetime(df_port['VSL_TIMESTAMP'])
    df_port = df_port.sort_values(by=['VSL_ID', 'TRIP', 'VSL_TIMESTAMP'], ascending=[True, True, False])
    
    # 대기 구간을 산출하고 새로운 컬럼에 담기
    waiting_data = []
    for vsl_id, vsl_group in df_port.groupby('VSL_ID'):
        for trip_id, trip_group in vsl_group.groupby('TRIP'):
            trip_group = trip_group.reset_index(drop=True)
            waiting_order = 0
            waiting_start = None
            lat, lon = None, None
            for i in range(len(trip_group)):
                if trip_group.loc[i, 'VSL_SPEED'] <= 3:
                    if waiting_start is None:
                        waiting_start=trip_group.loc[i, 'VSL_TIMESTAMP']
                        start_lat=trip_group.loc[i, 'LAT']
                        start_lon=trip_group.loc[i, 'LON']
                    waiting_end=trip_group.loc[i, 'VSL_TIMESTAMP']
                    end_lat=trip_group.loc[i, 'LAT']
                    end_lon=trip_group.loc[i, 'LON']
                    draft=trip_group.loc[i,'VSL_DRAFT']
                else:
                    if waiting_start is not None:
                        waiting_order += 1
                        waiting_time=waiting_start-waiting_end
                        waiting_data.append({
                            'VSL_ID': vsl_id,
                            'TRIP': trip_id,
                            'WAITING_ORDER': waiting_order,
                            'WAITING_START': waiting_end,
                            'WAITING_END': waiting_start,
                            'WAITING_TIME(H:M:S)': waiting_time,
                            'START_LAT': end_lat,
                            'START_LON': end_lon,
                            'END_LAT': start_lat,
                            'END_LON': start_lon,
                            'TO_PORT_CD': port_code,
                            'DRAFT':draft
                        })
                        waiting_start = None
                        start_lat, start_lon = None, None
                        end_lat, end_lon = None, None
            if waiting_start is not None:
                waiting_order += 1
                waiting_time=waiting_start-waiting_end
                waiting_data.append({
                    'VSL_ID': vsl_id,
                    'TRIP': trip_id,
                    'WAITING_ORDER': waiting_order,
                    'WAITING_START': waiting_end,
                    'WAITING_END': waiting_start,
                    'WAITING_TIME(H:M:S)': waiting_time,
                    'START_LAT': end_lat,
                    'START_LON': end_lon,
                    'END_LAT': start_lat,
                    'END_LON': start_lon,
                    'TO_PORT_CD': port_code,
                    'DRAFT':draft,
                })

    waiting_df = pd.DataFrame(waiting_data)
    all_waiting_data.append(waiting_df)
    # print(waiting_df['TO_PORT_CD'].unique())
    # print(waiting_df.info())
    # print('='*100)

# 모든 항구의 대기 데이터 통합 및 저장
final_waiting_df = pd.concat(all_waiting_data, ignore_index=True)
final_waiting_df = final_waiting_df[final_waiting_df['DRAFT'] != 999]
# print(final_waiting_df.info())
# display(final_waiting_df.head(1))
# final_waiting_df.to_csv('ACTUAL_WAITING_drop_outliers.csv', index=False)

final_waiting_df.to_csv('final_waiting_with_draft.csv', index=False)
print("END")
# final_waiting_df.info()

In [ ]:
# final_waiting_df = final_waiting_df[final_waiting_df['DRAFT'] != 999]
final_waiting_df
# final_waiting_df.shape

## 실질 대기에 대한 기술 통계

In [ ]:
final_waiting_df['WAITING_TIME_SECONDS'] = final_waiting_df['WAITING_TIME(H:M:S)'].apply(lambda x: pd.to_timedelta(x).total_seconds())

idx = final_waiting_df.groupby(['VSL_ID', 'TRIP'])['WAITING_TIME_SECONDS'].idxmax()
actual_waiting_df = final_waiting_df.loc[idx]

describe_actual_waiting = actual_waiting_df.groupby('TO_PORT_CD').agg({
    'WAITING_TIME_SECONDS': ['mean', 'min', 'max','median']
}).reset_index()

describe_actual_waiting.columns = ['TO_PORT_CD', 'AVG_ACTUAL_WAITING_TIME', 'MIN_WAITING_TIME', 'MAX_WAITING_TIME','MEDIAN_WAITING_TIME']

describe_actual_waiting['AVG_ACTUAL_WAITING_TIME'] = pd.to_timedelta(describe_actual_waiting['AVG_ACTUAL_WAITING_TIME'], unit='s')
describe_actual_waiting['MIN_WAITING_TIME'] = pd.to_timedelta(describe_actual_waiting['MIN_WAITING_TIME'], unit='s')
describe_actual_waiting['MAX_WAITING_TIME'] = pd.to_timedelta(describe_actual_waiting['MAX_WAITING_TIME'], unit='s')
describe_actual_waiting['MEDIAN_WAITING_TIME'] = pd.to_timedelta(describe_actual_waiting['MEDIAN_WAITING_TIME'], unit='s')

# describe_actual_waiting.to_csv('DESCRIBE_actual_waiting.csv_drop_outliers.csv', index=False)
print('END')

In [ ]:
describe_actual_waiting

# 군집화를 위한 데이터프레임 생성 (김정수)

## 원데이터에서 실질 대기 구역 내 모든 좌표 가져오기

In [ ]:
files = {
    'SGSIN': '2.2_SGSIN_drop_outlier.csv',
    'CNSHA': '2.2_CNSHA_drop_outlier.csv',
    'KRPUS': '2.2_KRPUS_drop_outlier.csv',
    'KRBNP': '2.2_KRBNP_drop_outlier.csv',
    'HKHKG': '2.2_HKHKG_drop_outlier.csv',
    'KRINC': '2.2_KRINC_drop_outlier.csv'
}

df_by_port = {port_code: pd.read_csv(file) for port_code, file in files.items()}
for port_code, df in df_by_port.items():
    df['VSL_TIMESTAMP'] = pd.to_datetime(df['VSL_TIMESTAMP'])
final_waiting_df = pd.read_csv('ACTUAL_WAITING_drop_outliers.csv', parse_dates=['WAITING_START', 'WAITING_END'])

# 실질 대기 구간 산출
final_waiting_df['WAITING_TIME_SECONDS']=final_waiting_df['WAITING_TIME(H:M:S)'].apply(lambda x: pd.to_timedelta(x).total_seconds()) # 비교를 위해 대기 시간 컬럼을 초단위로 통일
idx = final_waiting_df.groupby(['VSL_ID', 'TRIP'])['WAITING_TIME_SECONDS'].idxmax() # 최장 대기 구간이 actual_waiting임.
actual_waiting_df = final_waiting_df.loc[idx] # 최장대기 구간의 인덱스를 가져오기

coords_in_actual_waiting = [] # 원데이터에서 데이터 가져와 담을 리스트 생성
for port_code, port_df in df_by_port.items(): # 항구코드, 항구별 데이터
    # 대기 추출 데이터프레임에서 항구별 데이터 호출
    actual_waiting_subset=actual_waiting_df[actual_waiting_df['TO_PORT_CD']==port_code]
    # 행값 추출
    for _, row in actual_waiting_subset.iterrows():
        vsl_id=row['VSL_ID']
        waiting_time = row['WAITING_TIME(H:M:S)']
        draft=row['DRAFT']
        trip_id=row['TRIP']
        waiting_start=row['WAITING_START']
        waiting_end=row['WAITING_END']
        # 원데이터에서 실질 대기 구간 내 'TO_PORT_CD','VSL_TIMESTAMP', 'LAT', 'LON' 값 가져오기.
        records_in_actual_waiting=port_df[(port_df['VSL_ID']==vsl_id)&(port_df['TRIP']==trip_id)
                                &(port_df['VSL_TIMESTAMP']>=waiting_start)&(port_df['VSL_TIMESTAMP'] <= waiting_end)][['TO_PORT_CD','VSL_ID','TRIP','VSL_TIMESTAMP', 'LAT', 'LON']]
        records_in_actual_waiting['VSL_ID'] = vsl_id
        records_in_actual_waiting['WAITING_TIME'] = waiting_time
        records_in_actual_waiting['DRAFT']=draft
        coords_in_actual_waiting.append(records_in_actual_waiting)

coords_in_actual_waiting_df=pd.concat(coords_in_actual_waiting).reset_index(drop=True)
coords_in_actual_waiting_df['WAITING_TIME'] = pd.to_timedelta(coords_in_actual_waiting_df['WAITING_TIME'])
coords_in_actual_waiting_df.to_csv('ALL_COORDS_in_actual_waiting_with_draft.csv', index=False)
print('END')

In [ ]:
files = {
    'SGSIN': '2.2_SGSIN_drop_outlier.csv',
    'CNSHA': '2.2_CNSHA_drop_outlier.csv',
    'KRPUS': '2.2_KRPUS_drop_outlier.csv',
    'KRBNP': '2.2_KRBNP_drop_outlier.csv',
    'HKHKG': '2.2_HKHKG_drop_outlier.csv',
    'KRINC': '2.2_KRINC_drop_outlier.csv'
}

df_by_port = {port_code: pd.read_csv(file) for port_code, file in files.items()}
for port_code, df in df_by_port.items():
    df['VSL_TIMESTAMP'] = pd.to_datetime(df['VSL_TIMESTAMP'])
final_waiting_df = pd.read_csv('final_waiting_with_draft.csv', parse_dates=['WAITING_START', 'WAITING_END'])

# 실질 대기 구간 산출
final_waiting_df['WAITING_TIME_SECONDS']=final_waiting_df['WAITING_TIME(H:M:S)'].apply(lambda x: pd.to_timedelta(x).total_seconds()) # 비교를 위해 대기 시간 컬럼을 초단위로 통일
idx = final_waiting_df.groupby(['VSL_ID', 'TRIP'])['WAITING_TIME_SECONDS'].idxmax() # 최장 대기 구간이 actual_waiting임.
actual_waiting_df = final_waiting_df.loc[idx] # 최장대기 구간의 인덱스를 가져오기

coords_in_actual_waiting = [] # 원데이터에서 데이터 가져와 담을 리스트 생성
for port_code, port_df in df_by_port.items(): # 항구코드, 항구별 데이터
    # 대기 추출 데이터프레임에서 항구별 데이터 호출
    actual_waiting_subset=actual_waiting_df[actual_waiting_df['TO_PORT_CD']==port_code]
    # 행값 추출
    for _, row in actual_waiting_subset.iterrows():
        vsl_id=row['VSL_ID']
        waiting_time = row['WAITING_TIME(H:M:S)']
        draft=row['DRAFT']
        trip_id=row['TRIP']
        waiting_start=row['WAITING_START']
        waiting_end=row['WAITING_END']
        # 원데이터에서 실질 대기 구간 내 'TO_PORT_CD','VSL_TIMESTAMP', 'LAT', 'LON' 값 가져오기.
        records_in_actual_waiting=port_df[(port_df['VSL_ID']==vsl_id)&(port_df['TRIP']==trip_id)
                                &(port_df['VSL_TIMESTAMP']>=waiting_start)&(port_df['VSL_TIMESTAMP'] <= waiting_end)][['TO_PORT_CD','VSL_ID','TRIP','VSL_TIMESTAMP', 'LAT', 'LON']]
        records_in_actual_waiting['VSL_ID'] = vsl_id
        records_in_actual_waiting['WAITING_TIME'] = waiting_time
        records_in_actual_waiting['DRAFT']=draft
        coords_in_actual_waiting.append(records_in_actual_waiting)

coords_in_actual_waiting_df=pd.concat(coords_in_actual_waiting).reset_index(drop=True)
coords_in_actual_waiting_df['WAITING_TIME'] = pd.to_timedelta(coords_in_actual_waiting_df['WAITING_TIME'])
coords_in_actual_waiting_df.to_csv('ALL_COORDS_in_actual_waiting_with_draft.csv', index=False)
print('END')

In [ ]:
d=pd.read_csv('ALL_COORDS_in_actual_waiting_with_draft.csv')
d.shape

In [ ]:
files = {
    'SGSIN': '2.2_SGSIN_drop_outlier.csv',
    'CNSHA': '2.2_CNSHA_drop_outlier.csv',
    'KRPUS': '2.2_KRPUS_drop_outlier.csv',
    'KRBNP': '2.2_KRBNP_drop_outlier.csv',
    'HKHKG': '2.2_HKHKG_drop_outlier.csv',
    'KRINC': '2.2_KRINC_drop_outlier.csv'
}

df_by_port = {port_code: pd.read_csv(file) for port_code, file in files.items()}
for port_code, df in df_by_port.items():
    df['VSL_TIMESTAMP'] = pd.to_datetime(df['VSL_TIMESTAMP'])

# 'VSL_DRAFT'를 제외하고 날짜 형식의 열을 지정
final_waiting_df = pd.read_csv('final_waiting_with_draft.csv', parse_dates=['WAITING_START', 'WAITING_END'])

# 실질 대기 구간 산출
final_waiting_df['WAITING_TIME_SECONDS'] = final_waiting_df['WAITING_TIME(H:M:S)'].apply(lambda x: pd.to_timedelta(x).total_seconds())  # 비교를 위해 대기 시간 컬럼을 초단위로 통일
idx = final_waiting_df.groupby(['VSL_ID', 'TRIP'])['WAITING_TIME_SECONDS'].idxmax()  # 최장 대기 구간이 actual_waiting임.
actual_waiting_df = final_waiting_df.loc[idx]  # 최장대기 구간의 인덱스를 가져오기

coords_in_actual_waiting = []  # 원데이터에서 데이터 가져와 담을 리스트 생성
for port_code, port_df in df_by_port.items():  # 항구코드, 항구별 데이터
    # 대기 추출 데이터프레임에서 항구별 데이터 호출
    actual_waiting_subset = actual_waiting_df[actual_waiting_df['TO_PORT_CD'] == port_code]
    # 행값 추출
    for _, row in actual_waiting_subset.iterrows():
        vsl_id = row['VSL_ID']
        waiting_time = row['WAITING_TIME(H:M:S)']
        draft = row['DRAFT']
        trip_id = row['TRIP']
        waiting_start = row['WAITING_START']
        waiting_end = row['WAITING_END']
        # 원데이터에서 실질 대기 구간 내 'TO_PORT_CD','VSL_TIMESTAMP', 'LAT', 'LON' 값 가져오기.
        records_in_actual_waiting = port_df[
            (port_df['VSL_ID'] == vsl_id) & (port_df['TRIP'] == trip_id)
            & (port_df['VSL_TIMESTAMP'] >= waiting_start) & (port_df['VSL_TIMESTAMP'] <= waiting_end)
        ][['TO_PORT_CD', 'VSL_ID', 'TRIP', 'VSL_TIMESTAMP', 'LAT', 'LON']]
        records_in_actual_waiting['VSL_ID'] = vsl_id
        records_in_actual_waiting['WAITING_TIME'] = waiting_time
        records_in_actual_waiting['DRAFT'] = draft
        coords_in_actual_waiting.append(records_in_actual_waiting)

coords_in_actual_waiting_df = pd.concat(coords_in_actual_waiting).reset_index(drop=True)
coords_in_actual_waiting_df['WAITING_TIME'] = pd.to_timedelta(coords_in_actual_waiting_df['WAITING_TIME'])
coords_in_actual_waiting_df.to_csv('ALL_COORDS_in_actual_waiting_with_draft.csv', index=False)
print('END')

In [ ]:
s=pd.read_csv('ACTUAL_WAITING_drop_outliers.csv')
s.columns

# 대기 기간 내 중앙 좌표 추출하기 (곽현수, 유지민)
- 실질 대기 구역 ais 신호들을 사용

In [ ]:
import warnings

# 경고 메시지 필터링 설정
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

## 중앙 좌표 추출 (유지민) 
- 신호 데이터
- 1개 : 해당 좌표 return
- 2개 : 둘의 평균 좌표 return
- 3개 이상 : 해당 지점들의 중앙 좌표를 kmeans를 이요해 추출 

In [ ]:
def WAITING_POINT(wait_df):
    record = wait_df.shape[0]
    if record == 1:
        latitude = wait_df['LAT']
        longitude = wait_df['LON']
        return (latitude, longitude)
    elif record == 2:
        mid_lat = (wait_df.iloc[0]['LAT'] + wait_df.iloc[0]['LAT']) / 2
        mid_lon = (wait_df.iloc[1]['LON'] + wait_df.iloc[1]['LON']) / 2
        return (mid_lat, mid_lon)
    elif record >= 3:
        # KMeans 클러스터링 수행 (클러스터 수 K=1)
        data = wait_df[['LAT','LON']].to_numpy()
        kmean = KMeans(n_clusters=1, random_state=0)
        kmean.fit(data)
        # 중심점 추출
        center = kmean.cluster_centers_
        center_lat = center[0][0]
        center_lon = center[0][1]
        return (center_lat, center_lon)

## 실제 파일에 적용 (곽현수) 

In [ ]:
df = pd.read_csv('2.2_drop_outlier_waiting_before_arrive.csv.csv')
print(df.shape)
print(df.info())

In [ ]:
function_new_df = pd.DataFrame(columns=df.columns)

grouped_vsl = df.groupby('VSL_ID')

for vsl_id, group_vsl in grouped_vsl:
    grouped_trip = group_vsl.groupby('TRIP')
    for trip, group_trip in grouped_trip:
        
        coordinates = WAITING_POINT(group_trip)
        
        # 함수에서 도출한 좌표값을 function_new_df의 새로운 컬럼에 추가
        group_trip['WAIT_LAT'] = coordinates[0]
        group_trip['WAIT_LON'] = coordinates[1]
        
        function_new_df = pd.concat([function_new_df, group_trip], ignore_index=True)
        
print(function_new_df.head())

In [ ]:
display(function_new_df.head())

In [ ]:
function_new_df.to_csv('function_new_df.csv', index=False)